In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.metrics import confusion_matrix

In [2]:
full_data_w = [] #white wine data
full_data_r = [] #red whine data

def Pulldata(file):               #Pulling white wine data
    data_file = open(file, 'r')
    lines = data_file.readlines()
    temparr = []
    for k in lines:
        stuff = np.array(k.split(';'))
        temparr.append(stuff)    
    data_file.close()
    return(temparr)

full_data_w = Pulldata('winequality-white.csv')
full_data_r = Pulldata('winequality-red.csv')

#print(full_data_r)
#print(full_data_w)

In [3]:
#splitting data into train,validate and test data (60,20,20)%
WLabels = full_data_w[0]
del full_data_w[0]
RLabels = full_data_r[0]
del full_data_r[0]

WTrain = full_data_w[:round(len(full_data_w)*0.6)]
WValid = full_data_w[round(len(full_data_w)*0.6):round(len(full_data_w)*0.8)]
WTest  = full_data_w[round(len(full_data_w)*0.8):]

RTrain = full_data_r[:round(len(full_data_r)*0.6)]
RValid = full_data_r[round(len(full_data_r)*0.6):round(len(full_data_r)*0.8)]
RTest  = full_data_r[round(len(full_data_r)*0.8):]


print(len(WTrain))
print(len(WValid))
print(len(WTest))
print()
print(len(RTrain))
print(len(RValid))
print(len(RTest))



2939
979
980

959
320
320


In [4]:
#print(print(WTrain))

In [5]:
#Creating Design matrix : (DM) From Train Data


DMW = np.ones((len(WTrain),len(WLabels)+1))
DMR = np.ones((len(RTrain),len(RLabels)+1))


print(DMW.shape)
print(DMR.shape)

def CreateDM(DMtemp,DM):  #Function to create Design Matrix
    indexk = 0
    
    for k in DMtemp:
        indexj = 0
        for j in k:
            DM[indexk][indexj+1] = DMtemp[indexk][indexj]
            indexj=indexj+1
        indexk=indexk+1
    return(DM)        
        
DMW = CreateDM(WTrain , DMW) #Realocation of each designmatrix from ones to data

DMR = CreateDM(RTrain , DMR) #Realocation of each designmatrix from ones to data

DMW = np.delete(DMW, 12, 1)
DMR = np.delete(DMR, 12, 1)

#Creating Design Matrix for validation Data
DMWValid = np.ones((len(WValid),len(WLabels)+1))
DMRValid = np.ones((len(RValid),len(RLabels)+1))
   
DMWValid = CreateDM(WValid,DMWValid) #Realocation of each designmatrix from ones to data
DMRValid = CreateDM(RValid,DMRValid) #Realocation of each designmatrix from ones to data

DMWValid = np.delete(DMWValid, 12, 1)
DMRValid = np.delete(DMRValid, 12, 1)


#Create design matrix for Testing Data
DMWTest = np.ones((len(WTest),len(WLabels)+1))
DMRTest = np.ones((len(RTest),len(RLabels)+1))
      
DMWTest = CreateDM(WTest,DMWTest) #Realocation of each designmatrix from ones to data
DMRTest = CreateDM(RTest,DMRTest) #Realocation of each designmatrix from ones to data

DMWTest = np.delete(DMWTest, 12, 1)
DMRTest = np.delete(DMRTest, 12, 1)
 

(2939, 14)
(959, 14)


In [6]:
print(DMR)

[[ 1.    7.4   0.7  ...  0.56  9.4   1.  ]
 [ 1.    7.8   0.88 ...  0.68  9.8   1.  ]
 [ 1.    7.8   0.76 ...  0.65  9.8   1.  ]
 ...
 [ 1.    9.    0.36 ...  0.62 11.3   2.  ]
 [ 1.    9.5   0.37 ...  0.58 11.1   2.  ]
 [ 1.    6.4   0.57 ...  0.71 11.3   3.  ]]


In [7]:
def hFunction(DM,theta_values):
    return 1/(1+np.exp(-np.dot(DM,theta_values)))#H(x,Theta) Calculation

def GradientDescent(theta_values,theta_valuesOld,DM,alpha):#Gradient Descent Function
    while np.sqrt(np.sum(np.power(theta_values - theta_valuesOld, 2))) > 0.0005: # while euclidean norm > 0.0005 (so ϵ = 0.0005) 
        theta_valuesOld = theta_values # set old parameter values to parameter values before they are updated
        for i in range(DM.shape[0]):
            theta_values = theta_values - alpha*( (hFunction(DM[i,:12], theta_values) - DM[i][12]) * DM[i,:12] ) # update the parameters using the update rule

             
            
    return theta_values #return our theta values for this model

def convertMatrix(value,DM):  #Function to convert our design matrix for oneVsrest multiclass classification
    convDM = DM.copy()
    index = 0
    for k in DM:
        if k[12] == float(value):
            convDM[index][12] = 1
        else:
            convDM[index][12] = 0
        index = index+1
    return convDM

In [8]:
convertedDMR=convertMatrix(1,DMR)#create a converted design matrix where target variable only has two values 1 and 0 (1 represents the value you use as a parameter and 0 represents every other value besides the parameter value)
print(convertedDMR)


[[ 1.    7.4   0.7  ...  0.56  9.4   1.  ]
 [ 1.    7.8   0.88 ...  0.68  9.8   1.  ]
 [ 1.    7.8   0.76 ...  0.65  9.8   1.  ]
 ...
 [ 1.    9.    0.36 ...  0.62 11.3   0.  ]
 [ 1.    9.5   0.37 ...  0.58 11.1   0.  ]
 [ 1.    6.4   0.57 ...  0.71 11.3   0.  ]]


In [9]:

theta_values=np.random.uniform(size=12)#initialize our initial thetas
theta_valuesOld=np.zeros(12)
alpha=1e-4#learning rate
theta_values=GradientDescent(theta_values,theta_valuesOld,convertedDMR,alpha)
print(theta_values)#print our theta values for the model

[ 0.80237621  0.08408387  2.62396452  0.21128212  0.0175173   0.36747542
 -0.04384589  0.01848277  0.70932725  1.51976858 -1.1690032  -0.84657696]


In [10]:
print(DMR[0])
#DMRTest = DMRTest[DMRTest[:,1].argsort()] 
value=hFunction(DMRTest[:,:12],theta_values)#our predictions for a data point in terms of probability
print(value)



[ 1.      7.4     0.7     0.      1.9     0.076  11.     34.      0.9978
  3.51    0.56    9.4     1.    ]
[0.08779204 0.2237194  0.2237194  0.33108653 0.53718038 0.17523093
 0.23590364 0.0286233  0.06239636 0.50246599 0.50246599 0.3147919
 0.24195282 0.06032862 0.60715924 0.24195282 0.33561876 0.33561876
 0.08361252 0.09262496 0.87555932 0.12065558 0.33029506 0.12706867
 0.11427154 0.75249864 0.62916421 0.67310355 0.2753893  0.67310355
 0.56460446 0.62916421 0.04774372 0.76259759 0.37005766 0.34978273
 0.69673721 0.25149311 0.11270528 0.69673721 0.53409017 0.71958803
 0.24521486 0.09826054 0.1204278  0.21471571 0.21471571 0.21471571
 0.21471571 0.55686398 0.69281084 0.69281084 0.75328122 0.37694422
 0.70957961 0.62425864 0.06068102 0.43302915 0.43302915 0.43302915
 0.29550865 0.29550865 0.29550865 0.30547091 0.29550865 0.18816072
 0.31664974 0.21307302 0.59674335 0.59674335 0.25568171 0.46310317
 0.22527043 0.55379953 0.55379953 0.48476372 0.35212514 0.35708199
 0.23387557 0.47323033 

In [11]:
#loop for each possible quility /10

ThetaList = []
alpha=1e-4#learning rate
valuelist = []

for k in range(4):
    convertedDMR=convertMatrix(k,DMR)
    theta_values=np.random.uniform(size=12)#initialize our initial thetas
    theta_valuesOld=np.zeros(12)
    theta_values=GradientDescent(theta_values,theta_valuesOld,convertedDMR,alpha)
    ThetaList.append(theta_values)

#loop for h functions
for t in ThetaList:
    val = hFunction(DMRTest[:,:12],t)
    valuelist.append(val)


In [12]:
#soft max function
def softMax(ThetaList,TestItem):
    probabilities=[] #initialises probabilities array that holds probability for each class
    value=0
    for i in ThetaList: #loops through ThetaList 
        value=value+np.exp(np.dot(i,TestItem)) #calculates denominator of softmax function
    for j in ThetaList:
        probability=(np.exp(np.dot(j,TestItem)))/value #calculates probability of each class and appends it to the probabilties array
        probabilities.append(probability)
    return probabilities

DMRAns = DMRTest[:,12]
DMRtempTest = np.delete(DMRTest, 12, 1)
#print(DMRAns)
Guess = []
for k in range(len(DMRtempTest)):
    Guess.append(np.argmax(softMax(ThetaList,DMRtempTest[k])))
    
#print(Guess)



print(confusion_matrix(DMRAns,Guess))


[[112  33  14]
 [ 43  55  38]
 [  1   7  17]]


In [13]:
#print(ThetaList)
#Validate
index = 0
correct = 0
incorrect = 0
for k in DMRAns:
    if k == Guess[index]:
        correct=correct+1
    else:
        incorrect = incorrect+1
    index = index+1
print((correct/len(Guess)*100))

57.49999999999999


In [15]:
### After running this we have:
#ThetaList from the learning phase

DMRValidAns = DMRValid[:,12]
DMRtempValid = np.delete(DMRValid, 12, 1)
#print(DMRAns)
GuessV = []
for k in range(len(DMRtempValid)):
    GuessV.append(np.argmax(softMax(ThetaList,DMRtempValid[k])))
    
#print(Guess)



print(confusion_matrix(DMRValidAns,GuessV))

index = 0
correct = 0
incorrect = 0
for k in DMRValidAns:
    if k == GuessV[index]:
        correct=correct+1
    else:
        incorrect = incorrect+1
    index = index+1
print((correct/len(GuessV)*100))

[[62 23 13]
 [34 42 78]
 [ 2 12 54]]
49.375


# 